In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data=pd.read_csv('creditcard.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

#1 the time variable is based on second and range large, so here i first change the second into hour. if you think dont need to do this, just skip this step.

In [2]:
data.Time = data.Time.map(lambda x: divmod(x, 3600)[0])
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,0.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,0.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,0.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Data visualization and Feature Engineering, if you think there should be more things to talk about, just add on.

we can choose some of the features based on its distribution difference between class 0 and class 1.  and we remain the features which distribution have big different between class 0 and class 1.

In [3]:
cols = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

i drop 'V8', 'V13', 'V15', 'V20', 'V21' , 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28' based on the plot. 

In [4]:
cols = ['V8', 'V13', 'V15', 'V20', 'V21' , 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']
data.drop(columns=cols, inplace=True)
data.shape

(284807, 19)

Here i split the pd-training and pd-testing dataset by stratified split to remain the proportion of the class 0 and class 1 the same to make the results more reliable. if you consider the time series, i think it should be fine to just do random sampling, or just choose the last 0.2 as the test, should be OK. you can just combine the code to make a dataset you think you need, or just use different processing method to compare the results.

In [5]:
#stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit
X = np.array(data.loc[:,:'Amount'])
y = np.array(data["Class"])
sess = StratifiedShuffleSplit(n_splits = 5,test_size = 0.2,random_state = 0)

In [6]:
for train_index,test_index in sess.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [7]:
X_train.shape

(227845, 18)

In [8]:
X_test.shape

(56962, 18)

In [9]:
# if you want to scale the dataset, use the code below to scale the training and test dataset  (maybe you need to adjust the variable name to avoid error and fit your need)
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = preprocessing.StandardScaler().fit(X_train)  #this is to get the mean and std of training dataset
X_train=scaler.transform(X_train)  #scale the train
X_test=scaler.transform(X_test)  #scale the test using the mean and std of train

resampling: there are five resampling methods used. and you can use them to compare the result. here i use random forest, but actually different resampling methods lead to different results in different classification methods.  you can try them by yourself.

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RepeatedEditedNearestNeighbours

In [11]:
aos=ADASYN(random_state = 2)
X_aos, y_aos = aos.fit_sample(X_train, y_train)

In [12]:
X_aos.shape

(454933, 18)

In [13]:
kos = SMOTETomek(random_state = 2) 
X_kos, y_kos = kos.fit_sample(X_train, y_train)

In [14]:
X_kos.shape

(454902, 18)

In [15]:
sos = SMOTE(random_state = 2) 
X_sos, y_sos = sos.fit_sample(X_train, y_train)

In [16]:
X_sos.shape

(454902, 18)

In [17]:
smote_enn = SMOTEENN(random_state=2)
X_sot, y_sot = smote_enn.fit_sample(X_train, y_train)

In [18]:
X_sot.shape

(454518, 18)

In [19]:
renn = RepeatedEditedNearestNeighbours()
X_renn, y_renn= renn.fit_sample(X_train, y_train)

In [20]:
X_renn.shape

(227662, 18)

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,roc_auc_score,precision_score,roc_curve, recall_score, classification_report, f1_score,precision_recall_fscore_support)






SMOTETomek

In [22]:
#stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit
sess = StratifiedShuffleSplit(n_splits = 5,test_size = 0.005,random_state = 0)
for dispose, shrink in sess.split(X_kos, y_kos):
    X_kosd, X_koss = X_kos[dispose], X_kos[shrink]
    y_kosd, y_koss = y_kos[dispose], y_kos[shrink]

In [23]:
X_koss.shape

(2275, 18)

In [24]:
#smotetomek
train_datas=X_koss
train_targets=y_koss
train_data = X_kos
train_target = y_kos
test_target = y_test
test_data = X_test

In [25]:
train_targets.shape

(2275,)

In [26]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
import sklearn

sklearn.metrics.SCORERS.keys()

# Define our parameters to run a grid search over
parameters = {
    'penalty': ['l1','l2'],
    'C': [0.001,0.01,0.1,1,10,100,1000]
}

kfold = StratifiedKFold(n_splits=5)

gs = GridSearchCV(LogisticRegression(solver='saga'), param_grid=parameters, cv=kfold, scoring="roc_auc")
gs.fit(train_data, train_target)
print("Best auc: {}".format(gs.best_score_ ))
best_lr_params = gs.best_params_
print(gs.best_params_)

Best auc: 0.9854897348060551
{'C': 0.001, 'penalty': 'l2'}


Best auc: 0.9854897348060551
{'C': 0.001, 'penalty': 'l2'}

In [27]:
lr = LogisticRegression(**best_lr_params,solver='saga')
lr.fit(train_data,train_target)
test_est = lr.predict(test_data)
print("Logistic Regression accuracy:")
print(classification_report(test_target,test_est))
y_pred_gbc = lr.predict_proba(test_data)[:,1]
fpr_test, tpr_test, th_test = roc_curve(y_test,y_pred_gbc)
print('Logistic Regression AUC: %.4f' %auc(fpr_test, tpr_test))


Logistic Regression accuracy:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.89      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.55     56962
weighted avg       1.00      0.98      0.99     56962

Logistic Regression AUC: 0.9751


Logistic Regression accuracy:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.89      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.55     56962
weighted avg       1.00      0.98      0.99     56962

Logistic Regression AUC: 0.9751

In [28]:
cm = confusion_matrix(y_test, test_est)
cm

array([[55564,  1300],
       [   11,    87]], dtype=int64)

array([[55564,  1300],
       [   11,    87]], dtype=int64)

#smotetomek

Support Vector Machine (linear)

In [ ]:
#svm linear
from sklearn.svm import SVC
from sklearn import model_selection
svm = SVC(kernel = 'linear', C=1E10)
svm.fit(train_datas,train_targets)


In [ ]:
test_est1_svml = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est1_svml))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est1_svml)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm1_svml = confusion_matrix(y_test, test_est1_svml)
cm1_svml

In [ ]:
#svm rbf
svm = SVC(kernel = 'rbf', C=1E6)
svm.fit(train_datas,train_targets)
test_est1_svmr = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est1_svmr))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est1_svmr)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm1_svmr = confusion_matrix(y_test, test_est1_svmr)
cm1_svmr

In [ ]:
#svm poly
svm = SVC(kernel = 'poly', C=1E6)
svm.fit(train_datas,train_targets)
test_est1_svmp = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est1_svmp))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est1_svmp)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm1_svmp = confusion_matrix(y_test, test_est1_svmp)
cm1_svmp

SMOTE

In [29]:
#stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit
sess = StratifiedShuffleSplit(n_splits = 5,test_size = 0.005,random_state = 0)
for dispose,shrink in sess.split(X_sos, y_sos):
    X_sosd, X_soss = X_sos[dispose], X_sos[shrink]
    y_sosd, y_soss = y_sos[dispose], y_sos[shrink]

In [30]:
#smote
train_datas=X_soss
train_targets=y_soss
train_data = X_sos
train_target = y_sos
test_target = y_test
test_data = X_test

In [31]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

sklearn.metrics.SCORERS.keys()

# Define our parameters to run a grid search over
parameters = {
    'penalty': ['l1','l2'],
    'C': [0.001,0.01,0.1,1,10,100,1000]
}

kfold = StratifiedKFold(n_splits=5)

gs = GridSearchCV(LogisticRegression(solver='saga'), param_grid=parameters, cv=kfold, scoring="roc_auc")
gs.fit(train_data, train_target)
print("Best auc: {}".format(gs.best_score_ ))
best_lr_params = gs.best_params_
print(gs.best_params_)

Best auc: 0.9854897467898635
{'C': 0.001, 'penalty': 'l2'}


In [32]:
lr = LogisticRegression(**best_lr_params,solver='saga')
lr.fit(train_data,train_target)
test_est = lr.predict(test_data)
print("Logistic Regression accuracy:")
print(classification_report(test_target,test_est))
y_pred_gbc = lr.predict_proba(test_data)[:,1]
fpr_test, tpr_test, th_test = roc_curve(y_test,y_pred_gbc)
print('Logistic Regression AUC: %.4f' %auc(fpr_test, tpr_test))


Logistic Regression accuracy:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.89      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.55     56962
weighted avg       1.00      0.98      0.99     56962

Logistic Regression AUC: 0.9751


In [33]:
cm = confusion_matrix(y_test, test_est)
cm

array([[55564,  1300],
       [   11,    87]], dtype=int64)

SVM linear

In [ ]:
#svm linear
from sklearn.svm import SVC
from sklearn import model_selection
svm = SVC(kernel = 'linear', C=1E10)
svm.fit(train_datas,train_targets)
test_est2_svml = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est2_svml))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est2_svml)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm2_svml = confusion_matrix(y_test, test_est2_svml)
cm2_svml

In [ ]:
#svm rbf
svm = SVC(kernel = 'rbf', C=1E6)
svm.fit(train_datas,train_targets)
test_est2_svmr = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est2_svmr))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est2_svmr)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm2_svmr = confusion_matrix(y_test, test_est2_svmr)
cm2_svmr

In [ ]:
#svm poly
svm = SVC(kernel = 'poly', C=1E8)
svm.fit(train_datas,train_targets)
test_est2_svmp = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est2_svmp))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est2_svmp)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm2_svmp = confusion_matrix(y_test, test_est2_svmp)
cm2_svmp

SOMTEENN

In [34]:
#stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit
sess = StratifiedShuffleSplit(n_splits = 5,test_size = 0.005,random_state = 0)
for dispose, shrink in sess.split(X_sot, y_sot):
    X_sotd, X_sots = X_sot[dispose], X_sot[shrink]
    y_sotd, y_sots = y_sot[dispose], y_sot[shrink]

In [35]:
#SMOTEENN
train_datas = X_sots
train_targets = y_sots
train_data = X_sot
train_target = y_sot
test_target = y_test
test_data = X_test

In [36]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

sklearn.metrics.SCORERS.keys()

# Define our parameters to run a grid search over
parameters = {
    'penalty': ['l1','l2'],
    'C': [0.001,0.01,0.1,1,10,100,1000]
}

kfold = StratifiedKFold(n_splits=5)

gs = GridSearchCV(LogisticRegression(solver='saga'), param_grid=parameters, cv=kfold, scoring="roc_auc")
gs.fit(train_data, train_target)
print("Best auc: {}".format(gs.best_score_ ))
best_lr_params = gs.best_params_
print(gs.best_params_)

Best auc: 0.9856389651993815
{'C': 0.001, 'penalty': 'l2'}


In [37]:
lr = LogisticRegression(**best_lr_params,solver='saga')
lr.fit(train_data,train_target)
test_est = lr.predict(test_data)
print("Logistic Regression accuracy:")
print(classification_report(test_target,test_est))
y_pred_gbc = lr.predict_proba(test_data)[:,1]
fpr_test, tpr_test, th_test = roc_curve(y_test,y_pred_gbc)
print('Logistic Regression AUC: %.4f' %auc(fpr_test, tpr_test))


Logistic Regression accuracy:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.89      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.55     56962
weighted avg       1.00      0.98      0.99     56962

Logistic Regression AUC: 0.9755


In [38]:
cm = confusion_matrix(y_test, test_est)
cm

array([[55555,  1309],
       [   11,    87]], dtype=int64)

SVM linear

In [ ]:
#svm linear
from sklearn.svm import SVC
from sklearn import model_selection
svm = SVC(kernel = 'linear', C=1E10)
svm.fit(train_datas,train_targets)
test_est3_svml = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est3_svml))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est3_svml)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm3_svml = confusion_matrix(y_test, test_est3_svml)
cm3_svml

In [ ]:
#svm rbf
svm = SVC(kernel = 'rbf', C=1E6)
svm.fit(train_datas,train_targets)
test_est3_svmr = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est3_svmr))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est3_svmr)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm3_svmr = confusion_matrix(y_test, test_est3_svmr)
cm3_svmr

In [ ]:
#svm poly
svm = SVC(kernel = 'poly', C=1E8)
svm.fit(train_datas,train_targets)
test_est3_svmp = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est3_svmp))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est3_svmp)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm3_svmp = confusion_matrix(y_test, test_est3_svmp)
cm3_svmp

RENN

In [39]:
#stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit
sess = StratifiedShuffleSplit(n_splits = 5,test_size = 0.01,random_state = 0)
for dispose, shrink in sess.split(X_renn, y_renn):
    X_rennd, X_renns = X_renn[dispose], X_renn[shrink]
    y_rennd, y_renns = y_renn[dispose], y_renn[shrink]

In [40]:
#RENN
train_datas = X_renns
train_targets = y_renns
train_data = X_renn
train_target = y_renn
test_target = y_test
test_data = X_test

In [41]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

sklearn.metrics.SCORERS.keys()

# Define our parameters to run a grid search over
parameters = {
    'penalty': ['l1','l2'],
    'C': [0.001,0.01,0.1,1,10,100,1000]
}

kfold = StratifiedKFold(n_splits=5)

gs = GridSearchCV(LogisticRegression(max_iter=500,solver='saga'), param_grid=parameters, cv=kfold, scoring="roc_auc")
gs.fit(train_data, train_target)
print("Best auc: {}".format(gs.best_score_ ))
best_lr_params = gs.best_params_
print(gs.best_params_)

D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

Best auc: 0.978216004680234
{'C': 0.001, 'penalty': 'l2'}


In [42]:
lr = LogisticRegression(**best_lr_params,solver='saga')
lr.fit(train_data,train_target)
test_est = lr.predict(test_data)
print("Logistic Regression accuracy:")
print(classification_report(test_target,test_est))
y_pred_gbc = lr.predict_proba(test_data)[:,1]
fpr_test, tpr_test, th_test = roc_curve(y_test,y_pred_gbc)
print('Logistic Regression AUC: %.4f' %auc(fpr_test, tpr_test))


Logistic Regression accuracy:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.86      0.56      0.68        98

    accuracy                           1.00     56962
   macro avg       0.93      0.78      0.84     56962
weighted avg       1.00      1.00      1.00     56962

Logistic Regression AUC: 0.9720


D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [43]:
cm = confusion_matrix(y_test, test_est)
cm

array([[56855,     9],
       [   43,    55]], dtype=int64)

SVM linear

In [ ]:
#svm linear
from sklearn.svm import SVC
from sklearn import model_selection
svm = SVC(kernel = 'linear', C=1E10)
svm.fit(train_datas,train_targets)
test_est4_svml = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est4_svml))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est4_svml)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm4_svml = confusion_matrix(y_test, test_est4_svml)
cm4_svml

In [ ]:
#svm rbf
svm = SVC(kernel = 'rbf', C=1E6)
svm.fit(train_datas,train_targets)
test_est4_svmr = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est4_svmr))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est4_svmr)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm4_svmr = confusion_matrix(y_test, test_est4_svmr)
cm4_svmr

In [ ]:
#svm poly
svm = SVC(kernel = 'poly', C=1E8)
svm.fit(train_datas,train_targets)
test_est4_svmp = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est4_svmp))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est4_svmp)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm4_svmp = confusion_matrix(y_test, test_est4_svmp)
cm4_svmp

ADASYN

In [44]:
#stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit
sess = StratifiedShuffleSplit(n_splits = 5,test_size = 0.005,random_state = 0)
for dispose, shrink in sess.split(X_aos, y_aos):
    X_aosd, X_aoss = X_aos[dispose], X_aos[shrink]
    y_aosd, y_aoss = y_aos[dispose], y_aos[shrink]

In [45]:
#ADASYN
train_datas = X_aoss
train_targets = y_aoss
train_data = X_aos
train_target = y_aos
test_target = y_test
test_data = X_test

In [46]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

sklearn.metrics.SCORERS.keys()

# Define our parameters to run a grid search over
parameters = {
    'penalty': ['l1','l2'],
    'C': [0.001,0.01,0.1,1,10,100,1000]
}

kfold = StratifiedKFold(n_splits=5)

gs = GridSearchCV(LogisticRegression(solver='saga'), param_grid=parameters, cv=kfold, scoring="roc_auc")
gs.fit(train_data, train_target)
print("Best auc: {}".format(gs.best_score_ ))
best_lr_params = gs.best_params_
print(gs.best_params_)

Best auc: 0.9278620466845975
{'C': 0.001, 'penalty': 'l1'}


In [47]:
lr = LogisticRegression(**best_lr_params,solver='saga')
lr.fit(train_data,train_target)
test_est = lr.predict(test_data)
print("Logistic Regression accuracy:")
print(classification_report(test_target,test_est))
y_pred_gbc = lr.predict_proba(test_data)[:,1]
fpr_test, tpr_test, th_test = roc_curve(y_test,y_pred_gbc)
print('Logistic Regression AUC: %.4f' %auc(fpr_test, tpr_test))


Logistic Regression accuracy:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     56864
           1       0.02      0.94      0.03        98

    accuracy                           0.90     56962
   macro avg       0.51      0.92      0.49     56962
weighted avg       1.00      0.90      0.95     56962

Logistic Regression AUC: 0.9751


In [48]:
cm = confusion_matrix(y_test, test_est)
cm

array([[51108,  5756],
       [    6,    92]], dtype=int64)

In [ ]:
#svm linear
from sklearn.svm import SVC
from sklearn import model_selection
svm = SVC(kernel = 'linear', C=1E10)
svm.fit(train_datas,train_targets)
test_est5_svml = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est5_svml))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est5_svml)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm5_svml = confusion_matrix(y_test, test_est5_svml)
cm5_svml

In [ ]:
#svm rbf
svm = SVC(kernel = 'rbf', C=1E6)
svm.fit(train_datas,train_targets)
test_est5_svmr = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est5_svmr))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est5_svmr)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm5_svmr = confusion_matrix(y_test, test_est5_svmr)
cm5_svmr

In [ ]:
#svm poly
svm = SVC(kernel = 'poly', C=1E8)
svm.fit(train_datas,train_targets)
test_est5_svmp = svm.predict(test_data)
print("Support Vector Machine accuracy:")
print(classification_report(test_target,test_est5_svmp))
fpr_test, tpr_test, th_test = roc_curve(test_target, test_est5_svmp)
print('Support Vector Machine AUC: %.4f' %auc(fpr_test, tpr_test))

In [ ]:
cm5_svmp = confusion_matrix(y_test, test_est5_svmp)
cm5_svmp